In [1]:
import numpy as np
import h5py
import os, sys
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline
from scipy.signal import savgol_filter
import pandas as pd
sys.path.remove(sys.path[0])

sys.path.append('/Users/ajc/Core/codes/simulator/debug/ats/tools/utils/')
#sys.path.append('/Users/ajc/CODES/ats/tools/utils/')
import get_transect_data

sys.path.append('/Users/ajc/Core/PreProcessing/PyScript')
import readdata

In [8]:
path = "/Users/ajc/Core/simulations/barrow_icewedges/hydrologic_connection/ProjectionsP/"
outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Feb16_uniform_om/'

In [15]:
#Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'],['CESM_C30', 'CESM_C31']]
#Simulations = [['CESM_C10','CESM_C11'], ['CESM_C20', 'CESM_C21'], ['CESM_C30','CESM_C31','CESM_C32','CESM_C33'] ]# modified rain model

#Dec 15, 2020
#Simulations = [['CESM_C20','CESM_C21','CESM_C22'], ['CESM_C10B','CESM_C11B','CESM_C12B','CESM_C13B'],['CESM_C30','CESM_C31','CESM_C32','CESM_C33','CESM_C34'] ]# modified rain model, dessicated zone =0.1, longwaves

#Dec 22, 2020: ProjectionsO
#Simulations = [['CESM_C2%s'%i for i in range(3)], ['CESM_C1%s'%i for i in range(4)],['CESM_C3%s'%i for i in range(3)] ]# modified rain model, dessicated zone =0.1, longwaves

#Feb 16, 2021: ProjectionsP
Simulations = [['CESM_C2%s'%i for i in range(3)], ['CESM_C1%s'%i for i in range(4)],['CESM_C3%sX'%i for i in range(3)] ]# modified rain model, dessicated zone =0.1, longwaves



In [16]:
key_evap = 'surface-evaporative_flux'
to_day_year_mm = (86400 * 365) *1000 
to_day_mm = (86400) *1000
to_year = 365
def getSurfaceData(i,loc_flag_, pos_):
    Data = []
    #Time = []
    loc_flag = True
    #print (i,loc_flag_, pos_)
    for s in Simulations[i]:
        #print (i, s)
        infile = os.path.join(path,s,'visdump_surface_data.h5')
        dx=h5py.File(infile,'r')
        #print (dx.keys)
        
        if (loc_flag_):
            Dat11 = np.array(readdata.GetSurfVarFromVis(infile,key_evap,cellid=pos_)[key_evap])
            Dat11 = Dat11*to_day_mm
        else:
            Dat11 = np.array(readdata.GetSurfVarFromVis_All(infile,key_evap)[key_evap])
            #print ('A: ',Dat11.shape)
            Dat11 = Dat11.mean(axis=1)*to_day_mm
            #print ('B: ', Dat11.shape)
        a1 = len(Dat11)%73
        a2 = len(Dat11) - a1
        #print ('A:', a1,a2)

        Dat = Dat11[:a2]
        Data.append(Dat)
        #print ('HERE',a1,a2, Tim1.shape)
        
    d = dict()
    d[key_evap] = np.concatenate(Data)
    print ('Total len:', len(np.concatenate(Data)))
    
    return d

In [17]:
from multiprocessing import Pool
from functools import partial
num_processors = 3
pool=Pool(processes = num_processors)

def call_data(loc_flag,pos):
    #print (loc_flag, pos)
    global DataSurf1, DataSurf2, DataSurf3
    prod_x=partial(getSurfaceData, loc_flag_=loc_flag, pos_=pos)
    DataSurf1, DataSurf2, DataSurf3 = pool.map(prod_x,[i for i in range(len(Simulations))])
    #DataSurf1 = pool.map(prod_x,[i for i in range(len(Simulations))])
    Data = [DataSurf1, DataSurf2, DataSurf3]
    return Data

Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935
Total len: 6935


In [18]:
def get_summer_evap(d,evap_summer=0.5):
    x = []
    for dat in d:
        d1 = dat[dat>evap_summer] #if summer Evaporation is required than set evap_summer= 0.5
        x.append(np.mean(d1))
        
    #print (x)
    return x
    #d1 = d1.mean(axis=1)
    
def get_moving_average1(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < len1 - m:
            s1 = np.mean(d[i:i+m])
        else:
            s1 = np.mean(d[i:i+m])
        d_avg.append(s1)
    return d_avg

def get_moving_average(d, m=5):
    d_avg = []
    len1 = len(d)
    for i in range(len1):
        if i < m:
            s1 = np.mean(d[:i+1])
                
        else:
            s1= np.mean(d[i-m:i])

        d_avg.append(s1)
    return d_avg

In [19]:

def Plot_Evap():
    call_data(loc_flag=False, pos=0)
    fig, axs = plt.subplots(2,2, figsize=(6,7), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$']
    Colors = ['k','b','r','r']
    T = ['a','b','c','d']
    X = np.linspace(0,365,73)
    dry_year = 22
    wet_year =91
    avg_year = [27,44,76]
    Y = [wet_year,dry_year,avg_year[1]]
    for i in range(3):
        s = 73*Y[i]
        e = 73*(Y[i]+1)
        #print (np.mean(DataSurf1[key_evap][s:e]))
        if i == 0:
            print (len(X), len(DataSurf1[key_evap][s:e]))
            axs[i].plot(X,DataSurf1[key_evap][s:e],'k',label=Labels[0])
            axs[i].plot(X,DataSurf2[key_evap][s:e],'b',label=Labels[1])
            axs[i].plot(X,DataSurf3[key_evap][s:e],'r',label=Labels[2])
            axs[i].legend(loc='best',fontsize=8,labelspacing=0.1)
        else:
            axs[i].plot(X,DataSurf1[key_evap][s:e],'k')
            axs[i].plot(X,DataSurf2[key_evap][s:e],'b')
            axs[i].plot(X,DataSurf3[key_evap][s:e],'r')
        if i ==0 or i ==2:
            axs[i].set_ylabel('Evaporation [mm/day]',fontsize=12, fontweight='normal')
        if i == 2:
            axs[i].set_xlabel('Day of year',fontsize=12, fontweight='normal')
        axs[i].set_xlim((0, 365))
        axs[i].set_ylim((0, 7.01))
        axs[i].text(6.5,6.5,'(%s)'%T[i],fontsize=12, fontweight='normal')
        axs[i].set_yticks(np.arange(0,7,2))
        
    d = np.reshape(DataSurf1[key_evap],(-1,73))#.mean(axis=1)
    d = get_summer_evap(d)
    d_avg5yr = get_moving_average(d)
    print (i, len(d_avg5yr))
    xx = np.linspace(2005,2100,95)
    #axs[3].plot(xx,d,'k') #annual average
   
    axs[3].plot(xx,d_avg5yr,'k') #moving 5 year average
    
    d = np.reshape(DataSurf2[key_evap],(-1,73))#.mean(axis=1)
    d = get_summer_evap(d)
    d_avg5yr = get_moving_average(d)
    xx = np.linspace(2005,2100,96)
    #axs[3].plot(xx,d,'b')
    print ('HERE: ',len(xx), len(d_avg5yr))
    axs[3].plot(xx,d_avg5yr,'b')
    
    d = np.reshape(DataSurf3[key_evap],(-1,73))#.mean(axis=1)
    d = get_summer_evap(d)
    d_avg5yr = get_moving_average(d)
    #axs[3].plot(xx,d,'r')
    xx = np.linspace(2005,2100,95)
    print ('HERE1: ',len(xx), len(d_avg5yr))
    axs[3].plot(xx,d_avg5yr,'r') #moving 5 year average
    axs[3].text(2006,2.8,'(%s)'%T[3],fontsize=12, fontweight='normal')
    axs[3].set_xlim((2005, 2100))
    axs[3].set_ylim((0, 3.01))
    axs[3].set_yticks(np.arange(0,3.1,1))
    axs[3].set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    #plt.legend()
    #axs.set_ylim(135,138)
    
    #plt.savefig(outfile+'evaporation-center-moving5yr_avg.pdf', bbox_inches='tight', dpi=400)
    plt.show()

In [20]:
#Plot_Evap()

In [21]:
#forcing_polygon = h5py.File("/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8.5-2006-2100_dm1985-2018.h5",'r')
#forcing_polygon_new = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2018-feb2020.h5",'r')
#forcing_watershed = h5py.File("/Users/ajc/FUSE/simulations/production/barrow_watershed/data/CESM/CESM-RCP8_5-2006-2100_snowadjusted_supersmoothDM_2006.h5",'r')
forcing_watershed_FM = h5py.File("/Users/ajc/Core/PreProcessing/ats-intermediate-ngee/data/SNAP/CESM-RCP8.5-2006-2100_dm1985-2015-ET_2006.h5",'r')

In [22]:
"""
d1 = forcing_polygon['precipitation rain [m s^-1]']
d2 = forcing_polygon['precipitation snow [m SWE s^-1]']
rain_mm1 = np.reshape(d1,(-1,365)).mean(axis=1)*to_year_mm
snow_mm1 = np.reshape(d2,(-1,365)).mean(axis=1)*to_year_mm
precip_annual = snow_mm1 + rain_mm1


d1 = forcing_polygon_new['precipitation rain [m s^-1]']
d2 = forcing_polygon_new['precipitation snow [m SWE s^-1]']
rain_mm1 = np.reshape(d1,(-1,365)).mean(axis=1)*to_year_mm
snow_mm1 = np.reshape(d2,(-1,365)).mean(axis=1)*to_year_mm
precip_annual1 = snow_mm1 + rain_mm1

d1 = forcing_watershed ['precipitation rain [m s^-1]']
d2 = forcing_watershed ['precipitation snow [m SWE s^-1]']
rain_mm2 = np.reshape(d1,(-1,365)).mean(axis=1)*to_year_mm
snow_mm2 = np.reshape(d2,(-1,365)).mean(axis=1)*to_year_mm
precip_annual_watershed = snow_mm2 + rain_mm2
"""
d1 = forcing_watershed_FM ['precipitation rain [m s^-1]']
d2 = forcing_watershed_FM ['precipitation snow [m SWE s^-1]']
to_year_mm= 365*1000
rain_mm2 = np.reshape(d1,(-1,365)).mean(axis=1)*to_year_mm
snow_mm2 = np.reshape(d2,(-1,365)).mean(axis=1)*to_year_mm
precip_annual_watershed_FM = snow_mm2 + rain_mm2

In [23]:
#plt.plot(precip_annual,'k')
#plt.plot(precip_annual1,'g.')
#plt.plot(precip_annual_watershed,'r')
plt.plot(precip_annual_watershed_FM,'b')

In [24]:
%matplotlib qt
#outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Dec22_uniform_om/'
def Plot_Evap_Center_Trough_All():
    fig, axs = plt.subplots(3,1, figsize=(5,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    #Labels = [r'S$_{1}$', r'S$_{2}$', r'S$_{3}$']
    Colors = ['b','k','r','r']
    Labels = ['zero-head','specified-head','dynamic-head']
    T = ['Trough','Rim','Center','d']
    Avg_change = []
    for i in range(3):
        if i == 0:
            call_data(loc_flag=True,pos=0)
        elif (i==1):
            call_data(loc_flag=True,pos=11)
        else:
            call_data(loc_flag=True,pos=-1)
        if (i==10):
            print (len(DataSurf1[key_evap]),len(DataSurf2[key_evap]),len(DataSurf3[key_evap]))
            print (DataSurf1[key_evap],DataSurf2[key_evap],DataSurf3[key_evap])
        d = np.reshape(DataSurf1[key_evap],(-1,73)).mean(axis=1)*365
        d = get_summer_evap(d)
        d_avg5yr = get_moving_average(d,m=5)
        xx = np.linspace(2005,2100,95)
        #axs[3].plot(xx,d,'k') #annual average
        axs[i].plot(xx,d_avg5yr,'k',label=Labels[0]) #moving 5 year average
        x1 = (d[-1] - d[0])/d[0]*100
        Avg_change.append(x1)
        if (i==1):
            print (d_avg5yr)
        #----------------------------------------------------
        d = np.reshape(DataSurf2[key_evap],(-1,73)).mean(axis=1)*365
        d = get_summer_evap(d)
        d_avg5yr = get_moving_average(d)
        xx = np.linspace(2005,2100,95)
        #axs[3].plot(xx,d,'b')
        axs[i].plot(xx,d_avg5yr,'b',label=Labels[1])
    
        x1 = (d[-1] - d[0])/d[0]*100
        Avg_change.append(x1)
        
        #----------------------------------------------------
        d = np.reshape(DataSurf3[key_evap],(-1,73)).mean(axis=1)*365
        d = get_summer_evap(d)
        d_avg5yr = get_moving_average(d)
        xx = np.linspace(2005,2100,95)
        #axs[3].plot(xx,d,'r')
        x1 = (d[-1] - d[0])/d[0]*100
        Avg_change.append(x1)
        #---------------------------------------------------
        axs[i].plot(xx,d_avg5yr,'r',label=Labels[2]) #moving 5 year average
        if (i==0 or i ==2):
            axs[i].text(2086,10.,'%s'%T[i],fontsize=12, fontweight='normal')
        else:
            axs[i].text(2092,10.,'%s'%T[i],fontsize=12, fontweight='normal')
        axs[i].set_xlim((2005, 2100))
        #axs[i].set_ylim((0, 4))
        #axs[i].set_yticks(np.arange(0,4.1,1))
        if (i==2):
            axs[i].set_xlabel('Year',fontsize=12, fontweight='normal')
            
        axs[i].set_ylabel('Mean annual ET [mm]',fontsize=12, fontweight='normal')
        if (i==2):
            axs[2].legend(loc='upper left',labelspacing=.1,fontsize=12)
        #axs[2].legend(loc='upper right', fontsize=12, ncol=3, bbox_to_anchor=(0.7,-0.32, .1, .1))
        #axs.set_ylim(135,138)
        #--axs[2].plot(xx,precip_annual,'c',label=Labels[2]) #moving 5 year average
        axs[i].set_ylim(0,650)
        #axs[1].set_ylim(0,610)
        #axs[2].set_ylim(0,600)
        
    #print (Avg_change)
    plt.savefig(outfile+'evaporation-micro_loc-5yr_avg.pdf', bbox_inches='tight', dpi=500)
    plt.show()


In [25]:
Plot_Evap_Center_Trough_All()

[143.93392438732337, 97.31733944884573, 103.21766679255948, 93.70415322395779, 99.22157660802675, 99.22157660802675, 96.81457724409519, 108.42385538154288, 116.19085507252593, 128.87535968159023, 118.56222403498438, 118.54901750825813, 122.96684800226383, 108.24480437390818, 102.83376347835242, 128.15815971400758, 127.36158576506598, 121.99942879487526, 133.95171779701792, 146.69048607456944, 127.96789125422586, 110.8903758273113, 101.65282172706142, 97.23678236129008, 79.41661361347975, 78.1349345769557, 92.8826314237961, 102.04923711308936, 97.02257116926566, 113.76035321111658, 123.35974727419725, 128.6110782240537, 137.6312997053091, 133.8044474527457, 130.3978722604242, 125.12816568097453, 134.01696598655644, 143.42824312848896, 153.5451471017932, 173.21744797791956, 179.4831271517744, 163.73853789571336, 151.69931901887918, 157.58849225686694, 135.57484881031104, 137.57096039834926, 163.49959889240773, 171.2194368106794, 165.88830293847323, 170.33639813988668, 172.419831471434, 1

In [ ]:
#outfile='/Users/ajc/Documents/MyPapers/TroughBC/figures/Dec14/'
def Plot_AvgEvap_Precip_old():
    fig, axs = plt.subplots(1,2, figsize=(8,4), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Colors = ['b','k','r','r']
    Labels = ['zero head','specified head','dynamic head']
    T = ['a','b','c','d']
    Avg_change = []
    print ('HERE')
    call_data(loc_flag=False,pos=0)
    d = np.reshape(DataSurf1[key_evap],(-1,73)).mean(axis=1)*365
    d = get_summer_evap(d,evap_summer=0)
    d_avg5yr = get_moving_average(d,m=5)
    xx = np.linspace(2005,2100,95)
    axs[1].plot(xx,d_avg5yr,'k',label=Labels[0]) #moving 5 year average
    x1 = (d[-1] - d[0])/d[0]*100
    Avg_change.append(x1)
    f = np.polyfit(xx,d_avg5yr,1)
    p = np.poly1d(f)
    axs[1].plot(xx,p(xx),color='k',linestyle='dashed')
    print ('HERE1')
    #----------------------------------------------------
    d = np.reshape(DataSurf2[key_evap],(-1,73)).mean(axis=1)*365
    d = get_summer_evap(d,evap_summer=0)
    d_avg5yr = get_moving_average(d)
    xx = np.linspace(2005,2100,95)
    axs[1].plot(xx,d_avg5yr,'b',label=Labels[1])
    x1 = (d[-1] - d[0])/d[0]*100
    Avg_change.append(x1)
    f = np.polyfit(xx,d_avg5yr,1)
    p = np.poly1d(f)
    axs[1].plot(xx,p(xx),color='b',linestyle='dashed')
    print ('HERE2')
    #----------------------------------------------------
    d = np.reshape(DataSurf3[key_evap],(-1,73)).mean(axis=1)*365
    d = get_summer_evap(d,evap_summer=0)
    d_avg5yr = get_moving_average(d)
    xx = np.linspace(2005,2100,95)
    x1 = (d[-1] - d[0])/d[0]*100
    Avg_change.append(x1)
    f = np.polyfit(xx,d_avg5yr,1)
    p = np.poly1d(f)
    axs[1].plot(xx,p(xx),color='r',linestyle='dashed')
    #---------------------------------------------------
    axs[1].plot(xx,d_avg5yr,'r',label=Labels[2]) #moving 5 year average
    
    axs[1].set_xlim((2005, 2100))
    
    axs[1].set_xlabel('Calendar year',fontsize=12, fontweight='normal')

    axs[1].set_ylabel('Annual average evaporation [mm]',fontsize=12, fontweight='normal')
    axs[1].legend(loc='upper left',labelspacing=.1)
    axs[1].set_ylim(0,600)
    #print (Avg_change)
    
    #axs[2].legend(loc='upper right', fontsize=12, ncol=3, bbox_to_anchor=(0.7,-0.32, .1, .1))
    #axs.set_ylim(135,138)
    #--axs[2].plot(xx,precip_annual,'c',label=Labels[2]) #moving 5 year average
    axs[0].plot(xx,precip_annual_watershed_FM,'k',label=Labels[2]) #moving 5 year average
    
    f = np.polyfit(xx,precip_annual_watershed_FM,1)
    p = np.poly1d(f)
    axs[0].plot(xx,p(xx),color='k',linestyle='dashed')
    axs[0].set_ylim(0,600)
    axs[0].set_xlim((2005, 2100))
    axs[0].set_xlabel('Calendar year',fontsize=12, fontweight='normal')
    axs[0].set_ylabel('Annual average precipitation [mm]',fontsize=12, fontweight='normal')
    plt.tight_layout()
    #plt.savefig(outfile+'precip-evaporation-5yr_avg.pdf', bbox_inches='tight', dpi=500)
    plt.show()



In [ ]:
#Plot_AvgEvap_Precip()

In [26]:
%matplotlib qt
#outfile='/Users/ajc/Documents/MyPapers/TroughHydrodynamics/figures/Dec22_uniform_om/'
def Plot_AvgEvap_Precip():
    fig, axs = plt.subplots(1,1, figsize=(6,3.5), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1]})
    fig.subplots_adjust(hspace =.02, wspace=1.02)
    
    plt.tight_layout()
    #axs = axs.ravel()
    xpos = np.array([0, 1,2,3])
    Colors = ['b','k','r','r']
    Labels = ['zero head','specified head','dynamic head']
    T = ['a','b','c','d']
    Avg_change = []
    Data =call_data(loc_flag=False,pos=0)
    Data_temp = []
    for i in range(3):
        d = np.reshape(Data[i][key_evap],(-1,73)).mean(axis=1)*365
        d = get_summer_evap(d,evap_summer=0)
        d_avg5yr = get_moving_average(d,m=5)
        xx = np.linspace(2005,2100,95)
        axs.plot(xx,d_avg5yr,color=Colors[i],label=Labels[i]) #moving 5 year average
        f = np.polyfit(xx,d_avg5yr,1)
        p = np.poly1d(f)
        axs.plot(xx,p(xx),color=Colors[i],linestyle='dashed')
        Data_temp.append(d_avg5yr)
    axs.set_ylim(0,500)
    axs.set_xlim((2005, 2100))
    axs.set_xlabel('Year',fontsize=12, fontweight='normal')
    axs.set_ylabel('Mean annual ET [mm]',fontsize=12, fontweight='normal')
    axs.legend(loc='upper left',labelspacing=.1)
    plt.tight_layout()
    #plt.savefig(outfile+'ET-5yr_avg.pdf', bbox_inches='tight', dpi=500)
    Data_temp1 = np.reshape(Data_temp,(3,-1))
    np.savetxt(outfile+'ET-5yr_avg.dat', Data_temp1)
    plt.show()

In [27]:
Plot_AvgEvap_Precip()

In [ ]:
#d1 = forcing['precipitation rain [m s^-1]']
#d2 = np.reshape(d1,(-1,365)).mean(axis=1)*86400*365*1000

#print (min(d2),np.where(d2==min(d2)))
#print (max(d2),np.where(d2==max(d2)))

#x = d2[ (d2>191) & (d2<193)][0]
#print (x,d2[40],d2[76])
#print (np.mean(d2),np.where(d2==x))



In [ ]:
plt.plot(rain_mm,'r-*')
plt.plot(snow_mm,'g-*')
plt.plot(precip_annual,'k-*')
#plt.axhline(x,color='r')

In [ ]:
import re
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [ ]:
Simulations = ['CESM_C10','CESM_C11']
ss_data =h5py.File(os.path.join(path,Simulations[0],'visdump_data.h5'),'r')
wc = ss_data['water_content.cell.0']

In [ ]:
sf_data =h5py.File(os.path.join(path,Simulations[0],'visdump_surface_data.h5'),'r')
wc_sf = sf_data['surface-water_content.cell.0']
evap_sf = sf_data['surface-evaporative_flux.cell.0']

In [ ]:
Cycles = wc.keys()
Cycles = sorted_nicely(Cycles)
L = len(Cycles)%73
Cycles_orig = Cycles[:-L]
Cycles =  [Cycles[0], Cycles[72]]
print (Cycles[:10], len(Cycles))

In [ ]:
data_ss = []
for c in Cycles:
    d1 = np.concatenate(wc[c])
    d = np.sum(d1)
    data_ss.append(d)
#data_wc = np.reshape(data_ss,(-1,73))
data_wc = np.array(data_ss)

In [ ]:
data_sf = []
for c in Cycles:
    d1 = np.concatenate(wc_sf[c])
    d = np.sum(d1)
    data_sf.append(d)

#data_wc_sf = np.reshape(data_sf,(-1,73))
data_wc_sf = np.array(data_sf)

In [ ]:
data_sf = []
for c in Cycles_orig:
    d1 = np.concatenate(evap_sf[c])
    d = np.mean(d1)*to_year_mm
    data_sf.append(d)

#data_evap_sf = np.reshape(data_sf,(-1,73))
data_evap_sf = np.array(data_sf)

In [ ]:
data_wc.shape

In [ ]:
mass = (data_wc[-1] - data_wc[0]) / 55555.
mass_sf = (data_wc_sf[-1] - data_wc_sf[0])  / 55555.
area_sf = 61.92 #12 * 50
mass_total = mass + mass_sf
print (mass, mass_sf, mass_total/area_sf*1000) 

In [ ]:
print (precip_annual[0], np.mean(data_evap_sf))